# Running BPZliteEstimator with a custom set of Filters
authors: Sam Schmidt<br>
Last successfully run: June 14, 2024<br>


I have copied the filter files for HSC to rail_base FILTER

In [ ]:
from rail.utils.path_utils import RAILDIR
import os
custom_data_path = RAILDIR + '/rail/examples_data/estimation_data/data'
filterpath = RAILDIR + '/rail/examples_data/estimation_data/data/FILTER'


In [ ]:
!echo $tempbpzsedpath

This should have successfully copied the files to the proper SED directory. Now, we can proceed in the same manner that we did in the `BPZ_lite_demo.ipynb` notebook:

In [ ]:
import os
import qp
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import desc_bpz
from rail.core.data import TableHandle
from rail.core.stage import RailStage
from rail.estimation.algos.bpz_lite import BPZliteInformer, BPZliteEstimator

First, let's set up a DataStore, for more info on the DataStore, see the RAIL example notebooks:

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

First, let's grab the training and test data files that we will use in this example, they are included with RAIL, so we can access their location via the RAILDIR path.  Both file contain data drawn from the cosmoDC2_v1.1.4 truth extragalactic catalog generated by DESC with model 10-year-depth magnitude uncertainties.  The training data contains roughly 10,000 galaxies, while the test data contains roughly 20,000.  Both sets are representative down to a limiting apparent magnitude.

In [ ]:
#trainFile = os.path.join(RAILDIR, 'rail/examples_data/testdata/test_dc2_training_9816.hdf5')
testFile = "./dered_pdr3_wide_test_curated.hdf5"
test_data = DS.read_file("test_data", TableHandle, testFile)

In [ ]:
# set up lists of bands that we will use
bands = ['g','r','i','z','y']
mag_limits = [27.88, 27.05, 26.6, 26.6, 25.64]
usebands = []
useerrs = []
maglim_dict={}
for band, maglim in zip(bands, mag_limits):
    usebands.append(f"HSC{band}_cmodel_dered")
    useerrs.append(f"{band}_cmodel_magerr")
    maglim_dict[f"HSC{band}_cmodel_dered"] = maglim

Now, let's re-run BPZliteEstimator using this new prior and see if our results are any different:

In [ ]:
#nondetect_val=np.nan,
#    ref_band='HSCi_cmodel_dered',
#    redshift_col='specz_redshift',
#    mag_limits=maglim_dict,
#    zmax = 6.0,

In [ ]:
hdfnfile = os.path.join(RAILDIR, "rail/examples_data/estimation_data/data/CWW_HDFN_prior.pkl")
errmin=0.01
zperr=[]
for i in range(5):
    zperr.append(0.02)

In [ ]:
custom_dict = dict(hdf5_groupname="",
                   output="bpz_results_HSC.hdf5", 
                   prior_band='i_cmodel_magerr',
                   bands=usebands,
                   err_bands=useerrs,
                   columns_file="hsc_test.columns",
                   no_prior=False, ref_band = 'HSCi_cmodel_dered',
                   redshift_col='specz_redshift', mag_limits=maglim_dict,
                   zp_errors=zperr, mag_err_min=errmin,
                   zmax=4.0
                  )
custom_run = BPZliteEstimator.make_stage(name="hsc_bpz", **custom_dict, 
                                 model=hdfnfile)

Let's compute the estimate, and note that if this is the first time that you've run BPZ, you will see a bunch of lines print out as the code creates "AB" files (the model flux files used by BPZ and stored for later use) for the first time.

In [ ]:
%%time
custom_run.estimate(test_data)

In [ ]:
custom_res = qp.read("bpz_results_HSC.hdf5")


In [ ]:
custom_res.ancil

In [ ]:
sz = test_data()['specz_redshift']

And let's plot the modes fore this new run as well as our run with the default prior:

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(sz, custom_res.ancil['zmean'].flatten(), s=.0001, c='k', label='custom SED zmode')
plt.plot([0,3], [0,3], 'r--')
plt.xlabel("redshift")
plt.ylabel("photo-z mode")
plt.legend(loc='upper center', fontsize=10)
plt.xlim(-.005,4)
plt.ylim(-.005,4)

# Point estimate metrics

Let's see if our point estimate metrics have improved at all given the tuned prior.  These metrics take in arrays of the point estimates (we'll use the mode) and the true redshifts.

In [ ]:
from rail.evaluation.metrics.pointestimates import PointSigmaIQR, PointBias, PointOutlierRate, PointSigmaMAD

In [ ]:
custom_sigma_eval = PointSigmaIQR(custom_res.ancil['zmode'].flatten(), sz)

In [ ]:
custom_sigma = custom_sigma_eval.evaluate()

In [ ]:
print("custom SED sigma: %.4f" % (custom_sigma))

In [ ]:
custom_bias_eval = PointBias(custom_res.ancil['zmode'].flatten(), sz)

In [ ]:
custom_bias = custom_bias_eval.evaluate()
print("custom SED bias: %.4f" % (custom_bias))

In [ ]:
custom_outlier_eval = PointOutlierRate(custom_res.ancil['zmode'].flatten(), sz)

In [ ]:
custom_outlier = custom_outlier_eval.evaluate()
print("custom SED outlier rate: %.4f" % (custom_outlier))

In [ ]:
from rail.evaluation.metrics.pointestimates import PointStatsEz
custom_ez_eval = PointStatsEz(custom_res.ancil['zmode'].flatten(), sz)
custom_ez = custom_ez_eval.evaluate()
custom_outlier_frac = (np.sum((np.abs(custom_ez) > 0.15))) / len(sz)
print("fraction of catastrophic outliers: %.4f" % custom_outlier_frac)

Finally, we'll plot an example PDF.  Given that we are now comparing to a completely different set of SEDs with different predicted fluxes, we can expect different chi^2 values, and thus a complately different likelihood or posterior shape:

In [ ]:
whichone = 9515
fig, axs = plt.subplots(1,1, figsize=(10,6))
custom_res.plot_native(key=whichone, axes=axs, label="custom SED")
axs.set_xlabel("redshift")
axs.set_ylabel("PDF")
axs.legend(loc="upper center", fontsize=10)